Web Scrapping & Database connection

In [ ]:
#Enumerate function Code
#enumerate give us index for the data with the values
students=['Farhan','Abhishek','Abhinav','Tejashvi']
for index_values, data in enumerate(students):
    if 1<=index_values<=2:
        print(data)

In [130]:
import requests as r
from bs4 import BeautifulSoup
import pandas as pd

all_data=[] #used later

url="https://en.wikipedia.org/wiki/List_of_Indians_by_net_worth"
response=r.get(url)
html_data=response.content #returns html data from the url
soup=BeautifulSoup(html_data)
print('Done')


#soup.find_all('table',class='wikitable sortable jquery-tablesorter') #library couldn't find the content of this class
hello=soup.find_all('tr') #We want to find table row (tr) tag


#enumerate give us index for the data with the values
for index_no, value in enumerate(soup.find_all('tr')): 
    if 0<=index_no <=24: #Need rows from 0 to 24
        # print(index_no,value.text)
        # print(value.text.splitlines()) #This will give us empty data
        temp_data=value.text.splitlines()
        while '' in temp_data: #This will remove empty data/ blank spaces
            temp_data.remove('')
        # print(temp_data)
        all_data.append(temp_data)


print('Done')

#Now make a DataFrame of this list all_data
cols=all_data[0]
final_data=all_data[1:]
final_cols=['Rich_rank','Name','Net Worth','Company',"Source of Wealth",'None','None']

listing=[]
hello=len(final_data)
for i in range(1,hello+1):
    listing.append(i)


# rich_list_df=pd.DataFrame(final_data, index=listing ,columns=final_cols)
# rich_list_df.index.name="Rank"
# final_df=rich_list_df.drop(["Rank","None"], axis=1) #This will display the data too
# display(final_df)

rich_list_df=pd.DataFrame(final_data ,columns=final_cols)
#rich_list_df.index.name="Rank"
final_df=rich_list_df.drop(["None"], axis=1) #This will display the data too
display(final_df)

final_df.to_excel('Richest_People_in_India.xlsx') #export to excel file
print('Done')

Done
Done


,Rich_rank,Name,Net Worth,Company,Source of Wealth
0,1,Mukesh Ambani,116 billion,Reliance Industries,Diversified
1,2,Gautam Adani,84 billion,Adani Group,Diversified
2,3,Shiv Nadar,36.9 billion,HCL Technologies,IT services and consulting
3,4,Savitri Jindal,33.5 billion,JSW Group,Jindal Steel & Power
4,5,Dilip Shanghvi,26.7 billion,Sun Pharmaceutical Industries,Pharmaceuticals
5,6,Cyrus Poonawalla,21.3 billion,Serum Institute of India,Vaccines
6,7,Kushal Pal Singh,20.9 billion,DLF,Real estate
7,8,Kumar Mangalam Birla,19.7 billion,Aditya Birla Group,"Textiles, telecom, cement"
8,9,Radhakishan Damani,17.6 billion,"Avenue Supermarts, DMart","Investments, retail"
9,10,Lakshmi Mittal,16.4 billion,ArcelorMittal,Steel


Done


Now Build a SQL connection to send excel to database

In [131]:
import pymysql as py
conn=py.connect(user='root',
    password='1234',
    host='localhost',
    autocommit=True) #autosave
print('Connection created successfully')


cur=conn.cursor()


Connection created successfully


In [132]:
query='''create database wiki;'''
cur.execute(query)
print('Done')


query='''use wiki'''
cur.execute(query)



Done


0

Make columns of the table suitable for database


In [133]:
#we want all the column names in a list to make a column for a table
table_column=[i.replace(' ','_')  for i in final_df.columns] #replace space with _ to put this in database
#table_column is a list
query_part=' varchar(225), '.join(table_column) + ' varchar(225)'

Create Table rich_list

In [134]:
query='''Create table rich_list ('''+query_part+');'
print(query)
cur.execute(query)

Create table rich_list (Rich_rank varchar(225), Name varchar(225), Net_Worth varchar(225), Company varchar(225), Source_of_Wealth varchar(225));


0

To Close the Connection

In [ ]:
conn.close()

To Delete the Database

In [ ]:
import pymysql as py

# Establish the connection (you can reuse your existing connection object)
# Make sure to connect WITHOUT specifying the database you want to drop.
# Connect to the MySQL server itself.
conn = py.connect(user='root',
                  password='1234', # Replace with your password if different
                  host='localhost',
                  autocommit=True)

# Create a cursor
cur = conn.cursor()

# --- Code to Drop the Database ---
# The 'IF EXISTS' part prevents an error if the database doesn't exist.
query = '''DROP DATABASE IF EXISTS wiki;'''

try:
    cur.execute(query)
    print("Database 'wiki' dropped successfully (if it existed).")
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # It's good practice to close the connection when you're done
    conn.close()
